In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [ ]:
import os

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

from edm.real_time_ingestion.parse_register_schema import ParseRegisterSchema

In [ ]:
spark = SparkSession.builder.appName('').getOrCreate()
dbutils = DBUtils(spark)

In [ ]:
dbutils.widgets.text('file_path', '')
dbutils.widgets.text('file_name', '')
dbutils.widgets.text('spn_id', '')
dbutils.widgets.text('tenant_id', '')
dbutils.widgets.text('kv_name', '')
dbutils.widgets.text('adls_account_name', '')
dbutils.widgets.text('config_path', '')

In [ ]:
file_path = dbutils.widgets.get('file_path')
file_name = dbutils.widgets.get('file_name')
spn_id = dbutils.widgets.get('spn_id')
tenant_id = dbutils.widgets.get('tenant_id')
kv_name = dbutils.widgets.get('kv_name')
adls_account_name = dbutils.widgets.get('adls_account_name')
config_path = dbutils.widgets.get('config_path')

In [ ]:
# to be taken as ENV Variable
spn_password = os.getenv('SPN_PASSWORD', None)

In [ ]:
spn_credentials = dict()
spn_credentials['spn_id'] = spn_id
spn_credentials['spn_password'] = spn_password
spn_credentials['tenant_id'] = tenant_id

In [ ]:
parse_obj = ParseRegisterSchema(
        file_name, file_path, spark, dbutils,
        spn_credentials=spn_credentials, kv_name=kv_name,
        adls_account_name=adls_account_name, config_path=config_path
    )
parse_obj.run()